# Simple MNIST Handwritten digits detection

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.python.framework.graph_util import convert_variables_to_constants
import os

%matplotlib inline
mpl.rcParams['figure.dpi'] = 100

## Understand the MNIST dataset

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
batch_xs, batch_ys = mnist.train.next_batch(1)
img = batch_xs[0].reshape([28, 28])
label = batch_ys[0]

plt.imshow(img, cmap="gray")
print(label)

## Define Model

![title](images/mnist_two_layers.png)

In [ ]:
# 784 = 28x28 images
# None: We don't know how many items will be in this dimension
x = tf.placeholder(tf.float32, shape=[None, 784], name="input")
y_ = tf.placeholder(tf.float32, [None, 10])

# Weights and bias
# variables: we need to change this values when the model learns
fc1_W = tf.Variable(tf.truncated_normal([784, 10], stddev=0.1))
fc1_b = tf.Variable(tf.constant(0.1, shape=[10]))
fc1 = tf.nn.relu(tf.matmul(x, fc1_W) + fc1_b)

fc2_W = tf.Variable(tf.truncated_normal([10, 10], stddev=0.1))
fc2_b = tf.Variable(tf.constant(0.1, shape=[10]))

# define our model
y = tf.nn.softmax(tf.matmul(fc1, fc2_W) + fc2_b, name="output")

## Loss Function

In [ ]:
# loss
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

## Loss Optimization
![title](images/optimization.png)

In [ ]:
# loss optimization
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.5) \
    .minimize(cross_entropy)

## Learning Rate
### new_weight = existing_weight — learning_rate * gradient
![title](images/learning_rate.png)
![title](images/learning_rate_2.png)

## Define metrics

In [ ]:
# What is correct
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

# How accurate is it
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Train the model

In [ ]:
# Init all variables
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

# Train the model
num_steps = 3000
batch_size = 32
display_every = 200

for i in range(num_steps):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    if i % display_every == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch_xs, y_: batch_ys
        })
        print("Step {0}, training accuracy {1:.3f}%"
              .format(i, train_accuracy * 100))

## Validate results of the model

In [ ]:
# accuracy on test data
validation_accuracy = accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels
})

print("Validation Accuracy: {0:.3f}%".format(validation_accuracy * 100))

## Save model

In [ ]:
if not os.path.exists("models"):
    os.makedirs("models")

graph_path = "models/tensorflow-simple_mnist.pb"
graph_path_text = "models/tensorflow-simple_mnist.pb.txt"

minimal_graph = convert_variables_to_constants(sess, sess.graph_def, ["output"])
tf.train.write_graph(minimal_graph, '.', graph_path, as_text=False)
tf.train.write_graph(minimal_graph, '.', graph_path_text, as_text=True)
    
sess.close()

## Predict an image from saved model

### Load graph

In [ ]:
graph = tf.Graph()
graph_def = tf.GraphDef()

with open(graph_path, "rb") as f:
  graph_def.ParseFromString(f.read())
with graph.as_default():
  tf.import_graph_def(graph_def)

for op in graph.get_operations():
    print(str(op.name))

input_layer = "input"
output_layer = "output"

input_operation = graph.get_operation_by_name("import/" + input_layer)
output_operation = graph.get_operation_by_name("import/"+ output_layer)

### prepare image

In [ ]:
x = mnist.test.images[0]
print(x.shape)
x = np.expand_dims(x, axis=0)

# display image
img = x.reshape([28, 28])
print(img.shape)
plt.imshow(img, cmap="gray")

### Predict image

In [ ]:
with tf.Session(graph=graph) as session:
  results = session.run(output_operation.outputs[0], {
      input_operation.outputs[0]: x
  })

predictions = results[0]
print(predictions)
np.argmax(predictions)